### Since the Minimal Preprocessing approach did not work, here in this Notebook we'll do targeted error analysis by inspecting the data, if they are correctly labelled or not in the first place ###

In [8]:
import pandas as pd


#------- Loading the Dataset, and observing the Positive class -----------------
df = pd.read_csv("indian_liver_patient.csv")
df[df["Dataset"]==1].head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [9]:
#------Negatative Class ----------------

df[df["Dataset"] == 2].head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
8,17,Male,0.9,0.3,202,22,19,7.4,4.1,1.20,2
12,64,Male,0.9,0.3,310,61,58,7.0,3.4,0.90,2
15,25,Male,0.6,0.1,183,91,53,5.5,2.3,0.70,2
17,33,Male,1.6,0.5,165,15,23,7.3,3.5,0.92,2
24,63,Male,0.9,0.2,194,52,45,6.0,3.9,1.85,2


In [10]:
df["Dataset"] = df["Dataset"].map({1:1, 2:0})

In [11]:
#------ Dropping the duplicated rows-----------

df.drop_duplicates(inplace=True)

In [12]:
df["Dataset"].value_counts()

Dataset
1    406
0    164
Name: count, dtype: int64

# Data Understanding #

The normal range of Total Bilirubin in adults is typically:
 * Total Bilirubin: 0.3 to 1.2 mg/dL (milligrams per deciliter)

In [13]:
#----------- Inspecting Total Bilirubin ----------------------

total_bilirubin = df[(df["Total_Bilirubin"]<0.3) | (df["Total_Bilirubin"]>1.2)]
total_bilirubin

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1
5,46,Male,1.8,0.7,208,19,14,7.6,4.4,1.30,1
11,72,Male,2.7,1.3,260,31,56,7.4,3.0,0.60,1
...,...,...,...,...,...,...,...,...,...,...,...
574,32,Male,12.1,6.0,515,48,92,6.6,2.4,0.50,1
575,32,Male,25.0,13.7,560,41,88,7.9,2.5,2.50,1
576,32,Male,15.0,8.2,289,58,80,5.3,2.2,0.70,1
577,32,Male,12.7,8.4,190,28,47,5.4,2.6,0.90,1


<p>
    Above 1.2 mg/dL is considered elevated. [Mostly an Alarm]
</p>

In [8]:
total_bilirubin["Dataset"].value_counts()

Dataset
1    212
0     33
Name: count, dtype: int64

<p> There are 33 Records that are Normal despite having elevated Total_Bilirubin

In [14]:
tb_negative = total_bilirubin[total_bilirubin["Dataset"]==0]

<ul>
    <li>One cannot say these 33 rows are mislabeled just because Total_Bilirubin is abnormal.</li>
    <li>Elevated bilirubin ≠ guaranteed liver disease.</li>
    <li>Labels may be correct due to clinical context or other causes of elevation.</li>
</ul>

<p><b> Further Inspection is still required to come to the conclusion, if they are mislabelled or not.

In [15]:
#------ Inspecting Direct Bilirubin -----------------------------


direct_bilirubin = df[(df["Direct_Bilirubin"] < 0.1) | (df["Direct_Bilirubin"] > 0.3)]
direct_bilirubin["Dataset"].value_counts()

Dataset
1    228
0     40
Name: count, dtype: int64

<p> There are 40 Records that are having Direct_Bilirubin out side the range of Normal

In [17]:
db_negative = direct_bilirubin[direct_bilirubin["Dataset"]==0]

<p> Inspecting Total_Bilirubin and Direct_Bilirubin records intersect or not.

In [18]:
#--- Performing a kind of Inner Join betwee =n the two tables to come to the conclusion if the rows intersect or not

intersection_tb_db = pd.merge(tb_negative, db_negative, how="inner")
intersection_tb_db

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,33,Male,1.6,0.5,165,15,23,7.3,3.5,0.92,0
1,38,Female,2.6,1.2,410,59,57,5.6,3.0,0.80,0
2,42,Male,6.8,3.2,630,25,47,6.1,2.3,0.60,0
3,35,Male,1.8,0.6,275,48,178,6.5,3.2,0.90,0
4,70,Male,1.4,0.6,146,12,24,6.2,3.8,1.58,0
5,36,Male,5.3,2.3,145,32,92,5.1,2.6,1.00,0
6,50,Male,5.8,3.0,661,181,285,5.7,2.3,0.67,0
7,50,Male,7.3,3.6,1580,88,64,5.6,2.3,0.60,0
8,58,Male,1.7,0.8,188,60,84,5.9,3.5,1.40,0
9,60,Male,1.8,0.5,201,45,25,3.9,1.7,0.70,0


In [13]:
intersection_tb_db.shape

(33, 11)

<ul>
    <li>These 33 records are consistently abnormal in both features but still labeled 0 → likely borderline or subclinical cases, not mislabels.</li>
    <li>Since both indicators flag the same cases, they're likely highly correlated (as expected physiologically).</li>
    <li>Still, further analysis and evidence is required to come to the conclusion that they are correct/misclassified</li>

</ul>

In [19]:
#------- Inspecting Alkaline Phosphate ------------------

alp_abnormal = df[(df["Alkaline_Phosphotase"] < 40) | (df["Alkaline_Phosphotase"] > 130)]
alp_abnormal["Dataset"].value_counts()

Dataset
1    387
0    157
Name: count, dtype: int64

In [20]:
alp_negative = alp_abnormal[alp_abnormal["Dataset"]==0]

In [21]:
#---- Doing an innerJoin/Intersection between the AlkalinePhosphotise and already created intersection 
#-- table of TotalBilirubin and DirectBilirubin


intersection_tb_db_apl = pd.merge(alp_negative, intersection_tb_db, how="inner")

In [22]:
intersection_tb_db_apl

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,33,Male,1.6,0.5,165,15,23,7.3,3.5,0.92,0
1,38,Female,2.6,1.2,410,59,57,5.6,3.0,0.80,0
2,42,Male,6.8,3.2,630,25,47,6.1,2.3,0.60,0
3,35,Male,1.8,0.6,275,48,178,6.5,3.2,0.90,0
4,70,Male,1.4,0.6,146,12,24,6.2,3.8,1.58,0
5,36,Male,5.3,2.3,145,32,92,5.1,2.6,1.00,0
6,50,Male,5.8,3.0,661,181,285,5.7,2.3,0.67,0
7,50,Male,7.3,3.6,1580,88,64,5.6,2.3,0.60,0
8,58,Male,1.7,0.8,188,60,84,5.9,3.5,1.40,0
9,60,Male,1.8,0.5,201,45,25,3.9,1.7,0.70,0


<ul>
    <li>There are 31 non-diseased records with abnormal Total_Bilirubin, Direct_Bilirubin, and Alkaline_Phosphotase.</li>
    <li>These three features together suggest significant liver-related abnormality, yet they're still labeled as 0 (non-diseased).</li>
    <li>This strengthens the case that:
    <li>These 30 rows are either edge cases, label noise, or reflect undiagnosed/subclinical liver issues.</li>
    <li>Statistically, these are outliers among the “healthy” class and will likely confuse a classifier unless handled carefully.</li>
    <li><b>Still, I'll inspect one another medical indicator to further concrete the evidence.</li>

In [23]:
#------- Inspecting Alamine_Aminotransferase

alt_abnormal = df[(df["Alamine_Aminotransferase"] < 7) | (df["Alamine_Aminotransferase"] > 56)]
alt_abnormal["Dataset"].value_counts()

Dataset
1    139
0     18
Name: count, dtype: int64

In [24]:
#--- Segregating the Negative Cases

alt_abnormal_negative = alt_abnormal[alt_abnormal["Dataset"]==0]

In [25]:
#--- Performing Intersextion / Inner Join to find common rows

intersection_tb_db_apl_alt = pd.merge(alp_negative, intersection_tb_db_apl, how="inner")

In [26]:
intersection_tb_db_apl_alt.shape

(31, 11)

<h4>The Above Final Result concretes the evidence the Rows are Mislabelled as consistently 31 Rows despite having the abnormal values they were classified as Normal/No Liver Disease</h4>

In [22]:
intersection_tb_db_apl_alt

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,33,Male,1.6,0.5,165,15,23,7.3,3.5,0.92,0
1,38,Female,2.6,1.2,410,59,57,5.6,3.0,0.80,0
2,42,Male,6.8,3.2,630,25,47,6.1,2.3,0.60,0
3,35,Male,1.8,0.6,275,48,178,6.5,3.2,0.90,0
4,70,Male,1.4,0.6,146,12,24,6.2,3.8,1.58,0
5,36,Male,5.3,2.3,145,32,92,5.1,2.6,1.00,0
6,50,Male,5.8,3.0,661,181,285,5.7,2.3,0.67,0
7,50,Male,7.3,3.6,1580,88,64,5.6,2.3,0.60,0
8,58,Male,1.7,0.8,188,60,84,5.9,3.5,1.40,0
9,60,Male,1.8,0.5,201,45,25,3.9,1.7,0.70,0


In [23]:
pd.merge(alt_abnormal_negative, intersection_tb_db_apl, how="left")

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,64,Male,0.9,0.3,310,61,58,7.0,3.4,0.90,0
1,25,Male,0.6,0.1,183,91,53,5.5,2.3,0.70,0
2,38,Female,2.6,1.2,410,59,57,5.6,3.0,0.80,0
3,27,Male,1.2,0.4,179,63,39,6.1,3.3,1.10,0
4,50,Male,5.8,3.0,661,181,285,5.7,2.3,0.67,0
5,50,Male,7.3,3.6,1580,88,64,5.6,2.3,0.60,0
6,58,Male,1.7,0.8,188,60,84,5.9,3.5,1.40,0
7,38,Male,1.5,0.4,298,60,103,6.0,3.0,1.00,0
8,22,Male,2.7,1.0,160,82,127,5.5,3.1,1.20,0
9,61,Male,1.5,0.6,196,61,85,6.7,3.8,1.30,0


In [66]:
pd.merge(alt_abnormal_negative, intersection_tb_db_apl, how="right")

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,33,Male,1.6,0.5,165,15,23,7.3,3.5,0.92,0
1,38,Female,2.6,1.2,410,59,57,5.6,3.0,0.80,0
2,42,Male,6.8,3.2,630,25,47,6.1,2.3,0.60,0
3,35,Male,1.8,0.6,275,48,178,6.5,3.2,0.90,0
4,70,Male,1.4,0.6,146,12,24,6.2,3.8,1.58,0
5,36,Male,5.3,2.3,145,32,92,5.1,2.6,1.00,0
6,50,Male,5.8,3.0,661,181,285,5.7,2.3,0.67,0
7,50,Male,7.3,3.6,1580,88,64,5.6,2.3,0.60,0
8,58,Male,1.7,0.8,188,60,84,5.9,3.5,1.40,0
9,60,Male,1.8,0.5,201,45,25,3.9,1.7,0.70,0


In [29]:
pd.merge(alt_abnormal_negative, intersection_tb_db_apl, how="inner")

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,38,Female,2.6,1.2,410,59,57,5.6,3.0,0.80,0
1,50,Male,5.8,3.0,661,181,285,5.7,2.3,0.67,0
2,50,Male,7.3,3.6,1580,88,64,5.6,2.3,0.60,0
3,58,Male,1.7,0.8,188,60,84,5.9,3.5,1.40,0
4,38,Male,1.5,0.4,298,60,103,6.0,3.0,1.00,0
5,22,Male,2.7,1.0,160,82,127,5.5,3.1,1.20,0
6,61,Male,1.5,0.6,196,61,85,6.7,3.8,1.30,0
7,38,Male,2.2,1.0,310,119,42,7.9,4.1,1.00,0
8,39,Male,1.6,0.8,230,88,74,8.0,4.0,1.00,0


In [24]:
pd.merge(alt_abnormal_negative, intersection_tb_db_apl, how="outer")

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,4,Male,0.8,0.2,460,152,231,6.5,3.2,0.90,0
1,18,Male,1.3,0.7,316,10,21,6.0,2.1,0.50,0
2,19,Male,1.4,0.8,178,13,26,8.0,4.6,1.30,0
3,22,Male,0.8,0.2,300,57,40,7.9,3.8,0.90,0
4,22,Male,2.7,1.0,160,82,127,5.5,3.1,1.20,0
5,23,Female,2.3,0.8,509,28,44,6.9,2.9,0.70,0
6,24,Male,3.3,1.6,174,11,33,7.6,3.9,1.00,0
7,25,Male,0.6,0.1,183,91,53,5.5,2.3,0.70,0
8,26,Male,1.9,0.8,180,22,19,8.2,4.1,1.00,0
9,27,Male,1.2,0.4,179,63,39,6.1,3.3,1.10,0


<ul>
    <li>
        Interpretation:
        Now we have a solid cohort of non-diseased records that violate multiple independent liver function indicators.
    </li>
    <li>
        This isn't explainable by natural variation alone — they're either:

Subclinical cases (not labeled yet)

Mislabels

A limitation of the diagnostic label itself.
Even if medically correct, these rows are statistical anomalies in the Dataset = 0 class — and they will likely reduce your classifier’s confidence.
    </li>
</ul>



In [27]:
#--------Using all columns to match, assuming columns match exactly---------


df_clean = df.merge(intersection_tb_db_apl, how='outer', indicator=True)
df_clean = df_clean[df_clean['_merge'] == 'left_only'].drop(columns=['_merge'])

In [28]:
df_clean.shape

(539, 11)

In [29]:
df_clean["Dataset"].value_counts()

Dataset
1    406
0    133
Name: count, dtype: int64

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. Split

df_clean["Gender"] = df_clean["Gender"].map({"Male":1, "Female":0})

X = df_clean.drop(columns=["Dataset"])
y = df_clean["Dataset"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

# 2. Train with class weights
clf = RandomForestClassifier(class_weight='balanced', random_state=42)
clf.fit(X_train, y_train)

# 3. Predict & Evaluate
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))


[[ 8 19]
 [ 9 72]]
              precision    recall  f1-score   support

           0      0.471     0.296     0.364        27
           1      0.791     0.889     0.837        81

    accuracy                          0.741       108
   macro avg      0.631     0.593     0.600       108
weighted avg      0.711     0.741     0.719       108



In [29]:
df_clean.isnull().sum()

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    4
Dataset                       0
dtype: int64

In [30]:
df_clean[df_clean["Albumin_and_Globulin_Ratio"].isnull()]

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
88,27,Male,1.3,0.6,106,25,54,8.5,4.8,NaN,0
162,35,Female,0.6,0.2,180,12,15,5.2,2.7,NaN,0
270,45,Female,0.9,0.3,189,23,33,6.6,3.9,NaN,1
369,51,Male,0.8,0.2,230,24,46,6.5,3.1,NaN,1


In [31]:
#-----------Filling NaN in Albumin_and_Globulin_Ratio with class-wise median------------


df_clean["Albumin_and_Globulin_Ratio"] = df_clean.groupby("Dataset")["Albumin_and_Globulin_Ratio"]\
    .transform(lambda x: x.fillna(x.median()))

In [32]:
df_clean.isnull().sum()

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    0
Dataset                       0
dtype: int64

In [36]:
df_clean.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,4,Male,0.8,0.2,460,152,231,6.5,3.2,0.9,0
1,4,Male,0.9,0.2,348,30,34,8.0,4.0,1.0,0
2,6,Male,0.6,0.1,289,38,30,4.8,2.0,0.7,0
3,7,Female,27.2,11.8,1420,790,1050,6.1,2.0,0.4,1
4,7,Male,0.5,0.1,352,28,51,7.9,4.2,1.1,0


In [40]:
df_clean["Gender"] = df_clean["Gender"].map({
    "Male":1,
    "Female":0
})

In [41]:
from sklearn.model_selection import train_test_split

X = df_clean.drop(columns=["Dataset"])
y = df_clean["Dataset"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

In [42]:
from sklearn.model_selection import train_test_split

X = df_clean.drop(columns=["Dataset"])
y = df_clean["Dataset"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Before SMOTE:", X_train.shape, y_train.value_counts().to_dict())
print("After SMOTE:", X_train_resampled.shape, y_train_resampled.value_counts().to_dict())

Before SMOTE: (431, 10) {1: 325, 0: 106}
After SMOTE: (650, 10) {1: 325, 0: 325}


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[[18  9]
 [16 65]]
              precision    recall  f1-score   support

           0      0.529     0.667     0.590        27
           1      0.878     0.802     0.839        81

    accuracy                          0.769       108
   macro avg      0.704     0.735     0.714       108
weighted avg      0.791     0.769     0.777       108



In [44]:
from imblearn.combine import SMOTETomek

smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)

In [45]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_resampled, y_resampled)

y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[[15 12]
 [19 62]]
              precision    recall  f1-score   support

           0      0.441     0.556     0.492        27
           1      0.838     0.765     0.800        81

    accuracy                          0.713       108
   macro avg      0.640     0.660     0.646       108
weighted avg      0.739     0.713     0.723       108



In [46]:
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [47]:
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[[13 14]
 [16 65]]
              precision    recall  f1-score   support

           0      0.448     0.481     0.464        27
           1      0.823     0.802     0.812        81

    accuracy                          0.722       108
   macro avg      0.636     0.642     0.638       108
weighted avg      0.729     0.722     0.725       108



In [48]:
from xgboost import XGBClassifier

clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
clf.fit(X_resampled, y_resampled)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [49]:
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[[14 13]
 [17 64]]
              precision    recall  f1-score   support

           0      0.452     0.519     0.483        27
           1      0.831     0.790     0.810        81

    accuracy                          0.722       108
   macro avg      0.641     0.654     0.646       108
weighted avg      0.736     0.722     0.728       108



In [50]:
from imblearn.combine import SMOTEENN

In [51]:
# Apply SMOTEENN to training data
from collections import Counter
print("Before SMOTEENN:", Counter(y_train))

smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)

print("After SMOTEENN:", Counter(y_resampled))

Before SMOTEENN: Counter({1: 325, 0: 106})
After SMOTEENN: Counter({0: 235, 1: 189})


In [52]:
# Train model
clf = RandomForestClassifier(random_state=42)  # or XGBClassifier
clf.fit(X_resampled, y_resampled)

# Predict on test set
y_pred = clf.predict(X_test)

# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[[22  5]
 [28 53]]
              precision    recall  f1-score   support

           0      0.440     0.815     0.571        27
           1      0.914     0.654     0.763        81

    accuracy                          0.694       108
   macro avg      0.677     0.735     0.667       108
weighted avg      0.795     0.694     0.715       108



# Modeling #

In [53]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Before SMOTE:", X_train.shape, y_train.value_counts().to_dict())
print("After SMOTE:", X_train_resampled.shape, y_train_resampled.value_counts().to_dict())

Before SMOTE: (431, 10) {1: 325, 0: 106}
After SMOTE: (650, 10) {1: 325, 0: 325}


In [54]:
from sklearn.model_selection import GridSearchCV

In [57]:
from sklearn.ensemble import AdaBoostClassifier

In [58]:
ada_model = AdaBoostClassifier()

In [59]:
ada_model.fit(X_train_resampled, y_train_resampled)

AdaBoostClassifier()

In [60]:
# Predict on test set
y_pred = ada_model.predict(X_test)

# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[[20  7]
 [22 59]]
              precision    recall  f1-score   support

           0      0.476     0.741     0.580        27
           1      0.894     0.728     0.803        81

    accuracy                          0.731       108
   macro avg      0.685     0.735     0.691       108
weighted avg      0.790     0.731     0.747       108



In [51]:
import warnings

warnings.filterwarnings("ignore")

estimator = AdaBoostClassifier()

In [138]:
param_grid = {
    "n_estimators": [25,50,100,150,200,250,300,350,400],
    "learning_rate": [1.0,0.9, 0.5, 0.3, 0.1, 0.01, 0.001],
    "algorithm": ['SAMME', 'SAMME.R']
}

In [139]:
grid_ada = GridSearchCV(
    estimator,
    param_grid,
    cv=5,
    scoring="recall"
)
grid_ada.fit(X_train_resampled, y_train_resampled)

In [140]:
grid_ada.fit(X_train_resampled, y_train_resampled)

GridSearchCV(cv=5, estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [1.0, 0.9, 0.5, 0.3, 0.1, 0.01,
                                           0.001],
                         'n_estimators': [25, 50, 100, 150, 200, 250, 300, 350,
                                          400]},
             scoring='recall')

In [141]:
grid_ada.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 1.0, 'n_estimators': 350}

In [61]:
ada_model = AdaBoostClassifier(
    learning_rate=0.5,
    n_estimators=100
)

ada_model.fit(X_train_resampled, y_train_resampled)
# Predict on test set
y_pred = ada_model.predict(X_test)

# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[[19  8]
 [20 61]]
              precision    recall  f1-score   support

           0      0.487     0.704     0.576        27
           1      0.884     0.753     0.813        81

    accuracy                          0.741       108
   macro avg      0.686     0.728     0.695       108
weighted avg      0.785     0.741     0.754       108



In [62]:
ada_model = AdaBoostClassifier(
    learning_rate=1.0,
    n_estimators=200
)

ada_model.fit(X_train_resampled, y_train_resampled)
# Predict on test set
y_pred = ada_model.predict(X_test)

# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[[17 10]
 [16 65]]
              precision    recall  f1-score   support

           0      0.515     0.630     0.567        27
           1      0.867     0.802     0.833        81

    accuracy                          0.759       108
   macro avg      0.691     0.716     0.700       108
weighted avg      0.779     0.759     0.767       108



In [63]:
ada_model = AdaBoostClassifier(
    learning_rate=1.0,
    n_estimators=250
)

ada_model.fit(X_train_resampled, y_train_resampled)
# Predict on test set
y_pred = ada_model.predict(X_test)

# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[[17 10]
 [15 66]]
              precision    recall  f1-score   support

           0      0.531     0.630     0.576        27
           1      0.868     0.815     0.841        81

    accuracy                          0.769       108
   macro avg      0.700     0.722     0.709       108
weighted avg      0.784     0.769     0.775       108



In [65]:
df_clean.to_csv("cleaned_indian_liver_patient.csv")

In [66]:
df_clean.corr()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
Age,1.000000,0.061681,0.012122,0.007463,0.086371,-0.087092,-0.017076,-0.183756,-0.270956,-0.231292,0.146078
Gender,0.061681,1.000000,0.093465,0.105230,-0.022854,0.087854,0.084128,-0.086178,-0.095699,-0.011447,0.113361
Total_Bilirubin,0.012122,0.093465,1.000000,0.873536,0.202092,0.215751,0.237226,-0.007908,-0.226818,-0.207601,0.228117
Direct_Bilirubin,0.007463,0.105230,0.873536,1.000000,0.230041,0.235719,0.257064,-0.000575,-0.233935,-0.200977,0.256257
Alkaline_Phosphotase,0.086371,-0.022854,0.202092,0.230041,1.000000,0.126678,0.170731,-0.024231,-0.159671,-0.225237,0.210833
Alamine_Aminotransferase,-0.087092,0.087854,0.215751,0.235719,0.126678,1.000000,0.791672,-0.037945,-0.028795,-0.005609,0.157107
Aspartate_Aminotransferase,-0.017076,0.084128,0.237226,0.257064,0.170731,0.791672,1.000000,-0.023033,-0.086764,-0.073642,0.147399
Total_Protiens,-0.183756,-0.086178,-0.007908,-0.000575,-0.024231,-0.037945,-0.023033,1.000000,0.780198,0.230509,-0.063078
Albumin,-0.270956,-0.095699,-0.226818,-0.233935,-0.159671,-0.028795,-0.086764,0.780198,1.000000,0.683139,-0.192817
Albumin_and_Globulin_Ratio,-0.231292,-0.011447,-0.207601,-0.200977,-0.225237,-0.005609,-0.073642,0.230509,0.683139,1.000000,-0.187862


In [67]:
df_clean.skew()

Age                           -0.046989
Gender                        -1.143669
Total_Bilirubin                4.745263
Direct_Bilirubin               3.101217
Alkaline_Phosphotase           3.729986
Alamine_Aminotransferase       6.521044
Aspartate_Aminotransferase    10.298085
Total_Protiens                -0.327613
Albumin                       -0.057613
Albumin_and_Globulin_Ratio     1.019298
Dataset                       -1.178108
dtype: float64

In [68]:
df_clean.pivot_table(columns=df["Gender"], index=df["Dataset"])

Age            Alamine_Aminotransferase              Albumin  \
Gender      Female       Male                   Female       Male    Female   
Dataset                                                                       
0.0      49.304348  45.149533                84.956522  67.130841  3.150000   
1.0      45.647059  44.577855                62.094118  90.692042  3.118824   

                  Albumin_and_Globulin_Ratio           Alkaline_Phosphotase  \
Gender       Male                     Female      Male               Female   
Dataset                                                                       
0.0      3.183178                   0.948478  0.939720           321.326087   
1.0      3.130450                   0.910588  0.954256           315.023529   

                     ...   Dataset           Direct_Bilirubin            \
Gender         Male  ...    Female      Male           Female      Male   
Dataset              ...                                                  
0.0      286.570093  ...  0.826087  0.738318         2.089130  0.947664   
1.0      284.723183  ...  0.800000  0.733564         1.223529  1.708304   

           Gender           Total_Bilirubin           Total_Protiens            
Gender     Female      Male          Female      Male         Female      Male  
Dataset                                                                         
0.0      0.847826  0.738318        4.221739  2.887850       6.513043  6.605607  
1.0      0.823529  0.712803        2.587059  3.620069       6.460000  6.469896  

[2 rows x 22 columns]

In [69]:
df_clean.pivot_table(
    aggfunc="count",
    values="Gender",
    index=df["Dataset"]
)

,Gender
Dataset,
0.0,153
1.0,374


In [70]:
df_clean.pivot_table(
    aggfunc="skew",
    values=["Total_Bilirubin", "Direct_Bilirubin", "Albumin", "Albumin_and_Globulin_Ratio", "Direct_Bilirubin", "Total_Protiens"],
    index=["Dataset"]
)

,Albumin,Albumin_and_Globulin_Ratio,Direct_Bilirubin,Direct_Bilirubin,Total_Bilirubin,Total_Protiens
Dataset,,,,,,
0,-0.390814,0.559261,5.359686,5.359686,4.376309,-0.293520
1,0.042592,1.250416,2.608527,2.608527,4.131408,-0.326922


In [71]:
df_clean.pivot_table(
    aggfunc="skew",
    values=df_clean.columns,
    index=["Dataset"]
)

,Age,Alamine_Aminotransferase,Albumin,Albumin_and_Globulin_Ratio,Alkaline_Phosphotase,Aspartate_Aminotransferase,Direct_Bilirubin,Gender,Total_Bilirubin,Total_Protiens
Dataset,,,,,,,,,,
0,0.094366,3.756570,-0.390814,0.559261,2.451766,3.814981,5.359686,-0.691134,4.376309,-0.293520
1,-0.039700,5.676256,0.042592,1.250416,3.302568,9.056589,2.608527,-1.327948,4.131408,-0.326922


In [72]:
df_clean["AST_ALT_ratio"] = df_clean["Aspartate_Aminotransferase"] / df_clean["Alamine_Aminotransferase"]
df_clean["Bilirubin_ratio"] = df_clean["Direct_Bilirubin"] / df_clean["Total_Bilirubin"]
df_clean["Protein_gap"] = df_clean["Total_Protiens"] - df_clean["Albumin"]

In [73]:
df_clean.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset,AST_ALT_ratio,Bilirubin_ratio,Protein_gap
0,4,1,0.8,0.2,460,152,231,6.5,3.2,0.9,0,1.519737,0.250000,3.3
1,4,1,0.9,0.2,348,30,34,8.0,4.0,1.0,0,1.133333,0.222222,4.0
2,6,1,0.6,0.1,289,38,30,4.8,2.0,0.7,0,0.789474,0.166667,2.8
3,7,0,27.2,11.8,1420,790,1050,6.1,2.0,0.4,1,1.329114,0.433824,4.1
4,7,1,0.5,0.1,352,28,51,7.9,4.2,1.1,0,1.821429,0.200000,3.7


In [74]:
df_clean[["AST_ALT_ratio", "Bilirubin_ratio", "Protein_gap", "Dataset"]].corr()

,AST_ALT_ratio,Bilirubin_ratio,Protein_gap,Dataset
AST_ALT_ratio,1.000000,0.128556,0.164271,0.098420
Bilirubin_ratio,0.128556,1.000000,0.113201,0.188462
Protein_gap,0.164271,0.113201,1.000000,0.125855
Dataset,0.098420,0.188462,0.125855,1.000000


In [75]:
X = df_clean.drop("Dataset", axis=1)
y = df_clean["Dataset"]

In [76]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [77]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)


In [78]:
ada_model = AdaBoostClassifier(
    learning_rate=1.0,
    n_estimators=250
)

ada_model.fit(X_train_res, y_train_res)
# Predict on test set
y_pred = ada_model.predict(X_test)

# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[[15 12]
 [19 62]]
              precision    recall  f1-score   support

           0      0.441     0.556     0.492        27
           1      0.838     0.765     0.800        81

    accuracy                          0.713       108
   macro avg      0.640     0.660     0.646       108
weighted avg      0.739     0.713     0.723       108



In [80]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: Drop engineered features
df_model = df_clean.drop(columns=['AST_ALT_ratio', 'Bilirubin_ratio', 'Protein_gap'], errors='ignore')

# Step 2: Split features and target
X = df_model.drop('Dataset', axis=1)
y = df_model['Dataset']

# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Step 4: Apply SMOTE to balance training data
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

# Step 5: Train AdaBoost
model = AdaBoostClassifier(learning_rate=1, n_estimators=250, random_state=42)
model.fit(X_train_sm, y_train_sm)

# Step 6: Evaluate
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))


[[17 10]
 [15 66]]
              precision    recall  f1-score   support

           0      0.531     0.630     0.576        27
           1      0.868     0.815     0.841        81

    accuracy                          0.769       108
   macro avg      0.700     0.722     0.709       108
weighted avg      0.784     0.769     0.775       108



In [81]:
# 2. Extract importances into a DataFrame
importances = pd.Series(model.feature_importances_, index=X_train_sm.columns)
importances.sort_values(ascending=False)

Total_Bilirubin               0.164
Albumin_and_Globulin_Ratio    0.148
Alkaline_Phosphotase          0.124
Age                           0.120
Albumin                       0.108
Alamine_Aminotransferase      0.096
Direct_Bilirubin              0.080
Aspartate_Aminotransferase    0.080
Total_Protiens                0.080
Gender                        0.000
dtype: float64

In [82]:
import numpy as np

importances = {
    "Total_Bilirubin":               0.164,
    "Albumin_and_Globulin_Ratio":    0.148,
    "Alkaline_Phosphotase":          0.124,
    "Age":                           0.120,
    "Albumin":                       0.108,
    "Alamine_Aminotransferase":      0.096,
    "Direct_Bilirubin":              0.080,
    "Aspartate_Aminotransferase":    0.080,
    "Total_Protiens":                0.080,
    "Gender":                        0.000
}

# 1. Compute mean importance
threshold = np.mean(list(importances.values()))
print(f"Mean importance threshold = {threshold:.3f}")


Mean importance threshold = 0.100


In [83]:
selected = [
    "Total_Bilirubin",
    "Albumin_and_Globulin_Ratio",
    "Alkaline_Phosphotase",
    "Age",
    "Albumin"
]

X_train_sel = X_train_sm[selected]
X_test_sel  = X_test[selected]

In [84]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Initialize and train
ada_sel = AdaBoostClassifier(learning_rate=1, n_estimators=250, random_state=42)
ada_sel.fit(X_train_sel, y_train_sm)

# Predict and evaluate
y_pred_sel = ada_sel.predict(X_test_sel)
print(confusion_matrix(y_test, y_pred_sel))
print(classification_report(y_test, y_pred_sel, digits=3))


[[17 10]
 [16 65]]
              precision    recall  f1-score   support

           0      0.515     0.630     0.567        27
           1      0.867     0.802     0.833        81

    accuracy                          0.759       108
   macro avg      0.691     0.716     0.700       108
weighted avg      0.779     0.759     0.767       108



## Since AdaBoostClassifier is giving better results than others, in next phase we'll try to improve its performance further by prioriticing the recall ##